<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/test_runs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Welcome to Okareo!

Get your API token from [https://app.okareo.com/](https://app.okareo.com/) and set it in the cell below. 👇
   (Note: You will need to register first.)



In [ ]:
OKAREO_API_KEY = "<YOUR-API-TOKEN>"

%pip install okareo

In [ ]:
# perform a test run using a scenario set
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, SeedData, ScenarioType
from okareo.model_under_test import CustomModel

okareo = Okareo(OKAREO_API_KEY)

class ClassificationModel(CustomModel):
    # Callable to be applied to each scenario in the scenario set
    def invoke(self, input: str):
        # call your model being tested here using <input> from the scenario set
        if "how much" in input:
            actual = "pricing"
        elif "return" in input:
            actual = "returns"
        else:
            actual = "complaints"
        return actual, {"labels": actual, "confidence": .99 }  # return a tuple of (actual, overall model response context)

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="intent_classifier", model=ClassificationModel(name="Classification model"))

# generate example scenario based on seed data and return results in one call
scenario_set_create = ScenarioSetCreate(name="My Test Scenario Set",
                                        seed_data=[SeedData(input_="I want to send this product back", result="returns"),
                                                   SeedData(input_="my product is not working", result="complaints"),
                                                   SeedData(input_="how much is the warranty on the product", result="pricing"),
                                                   SeedData(input_="this product is having issues", result="complaints"),
                                                   SeedData(input_="I want to send this product back for a return", result="returns"),
                                                   SeedData(input_="how much is this product", result="pricing")])

scenario = okareo.create_scenario_set(scenario_set_create)
scenario_id = scenario.scenario_id

# use the scenario or scenario id to run the test
test_run_item = model_under_test.run_test(scenario=scenario_id, name="Intent Classifier Run", calculate_metrics=True)

# link back to Okareo site for evaluation visualization
model_results = test_run_item.model_metrics.to_dict()
app_link = test_run_item.app_link
print(f"See results in Okareo: {app_link}")

